# New Notes:

- Forked https://www.kaggle.com/yeayates21/xlm-roberta-augmentation-ssl-0-9417-pub-lb
- Added https://www.kaggle.com/sauravmaheshkar/huggingface-bert-variants so we can run without internet
- Revised to train using data https://www.kaggle.com/julian3833/jigsaw-toxic-comment-classification-challenge
- Revised to score Jigsaw Severity test set
- Running on DistilBERT to reduce memory usage for transition from TPU to GPU

# Old Notes:

**Kaggle Sources**
 - https://www.kaggle.com/xhlulu/jigsaw-tpu-xlm-roberta
     - forked this notebook
 - https://www.kaggle.com/shonenkov/tpu-training-super-fast-xlmroberta
     - grabed external data from this notebook, i.e. https://www.kaggle.com/shonenkov/open-subtitles-toxic-pseudo-labeling
     
**External Sources and More Additions**
 - Used [eda_nlp](https://github.com/jasonwei20/eda_nlp) to create an augmented version of the unintended bias dataset, then downsampled this data to have balanced dataset.  More information can be found here (minus the downsampling):  https://www.kaggle.com/yeayates21/jigsaw-bias-toxicity-eda-nlp-aug16-alpha005
 - pickled encoded data for faster runtime
 - some light manual hyperparameter tuning
 - scored the test set with each "model.fit" run as "checkpoint predictions" and blended the checkpoint predictions (I didn't checkpoint the models, but that could be easily added).

-----------------------------------------------------------------------

#### Acknowledgements

 - [@alexshonenkov](https://www.kaggle.com/shonenkov)
 - [@xhlulu](https://www.kaggle.com/xhlulu)


In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from kaggle_datasets import KaggleDatasets
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
import pickle
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

# Helper Functions

In [ ]:
def build_model(transformer, max_len=512):
    """
    https://www.kaggle.com/xhlulu/jigsaw-tpu-distilbert-with-huggingface-and-keras
    """
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=0.000009), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Configs

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 192
MODEL = '../input/huggingface-bert-variants/distilbert-base-uncased/distilbert-base-uncased'

# Tokenizer

In [ ]:
# First load the real tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL)

# Load Data

In [ ]:
!ls /kaggle/input/jigsawtpuxlmrobertacopypickledata

In [ ]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, # text to tokenize
        return_token_type_ids=False, # https://huggingface.co/transformers/glossary.html#token-type-ids
        pad_to_max_length=True, # add padding
        max_length=maxlen # set max length
    )
    
    return np.array(enc_di['input_ids'])

In [ ]:
df = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/train.csv")

toxic = 1.0
severe_toxic = 2.0
obscene = 2.0
threat = 1.0
insult = 1.0
identity_hate = 2.0

df['y'] = toxic*df['toxic'] +  severe_toxic*df['severe_toxic'] + obscene*df['obscene'] + threat*df['threat'] + insult*df['insult'] + identity_hate*df['identity_hate']
df = df[['comment_text', 'y']].rename(columns={'comment_text': 'text'})

#undersample non toxic comments  on Toxic Comment Classification Challenge

min_len = (df['y'] >= 1).sum()
df_y0_undersample = df[df['y'] == 0].sample(n=min_len, random_state=201)
df = pd.concat([df[df['y'] >= 1], df_y0_undersample])
df['y'].value_counts()

In [ ]:
X = df['text'].values
y = df['y'].values

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
PATH = '/kaggle/input/jigsaw-toxic-severity-rating/'
comment_data = pd.read_csv(PATH + 'comments_to_score.csv')
sub = pd.read_csv(PATH + 'sample_submission.csv')

In [ ]:
%%time

x_train = regular_encode(X_train.tolist(), tokenizer, maxlen=MAX_LEN)
x_valid = regular_encode(X_valid.tolist(), tokenizer, maxlen=MAX_LEN)
x_test = regular_encode(comment_data['text'].values.tolist(), tokenizer, maxlen=MAX_LEN)

# TF Datasets

In [ ]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

# Load Model

In [ ]:
%%time
with strategy.scope():
    transformer_layer = TFAutoModel.from_pretrained(MODEL)
    model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()

# Train

#### Train epoch 1

In [ ]:
n_steps = x_train.shape[0] // BATCH_SIZE

model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=1
)

checkpointPredictions1 = model.predict(test_dataset, verbose=1)
print(checkpointPredictions1[:10])

#### Train epoch 2

In [ ]:
model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=1
)

checkpointPredictions2 = model.predict(test_dataset, verbose=1)
print(checkpointPredictions1[:10])

#### Train epoch 3

In [ ]:
model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=1
)

checkpointPredictions3 = model.predict(test_dataset, verbose=1)
print(checkpointPredictions1[:10])

#### Train epoch 4

In [ ]:
model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=1
)

checkpointPredictions4 = model.predict(test_dataset, verbose=1)
print(checkpointPredictions1[:10])

# Submission

In [ ]:
sub = pd.read_csv('../input/jigsaw-toxic-severity-rating/sample_submission.csv')
sub['score'] = (checkpointPredictions1*0.05)+(checkpointPredictions2*0.10)+(checkpointPredictions3*0.76)+(checkpointPredictions4*0.09)
sub['score'] = sub['score'].rank(method='first')
sub.to_csv('submission.csv', index=False)

In [ ]:
plt.hist(sub.score.values)
plt.show()

In [ ]:
sub.head(30)